In [67]:
import numpy as np
import GestionDonnees as gd

In [68]:
parametres = {}
parametres['regression_logistique'] = {'C': [8, 10, 15, 20, 25],
                                       'solver': ['newton-cg', 'lbfgs', 'liblinear']}

In [69]:
donnees = gd.GestionDonnees(True, False)

In [70]:
X_entrain, y_entrain, x_test, y_test, x_nouveau_test = donnees.generer_donnees()

In [71]:
from test import classifier_donnees

In [73]:
from sklearn.linear_model import LogisticRegression
regression_logistique = LogisticRegression(max_iter=1000)

In [74]:
resultat_regression_logistique = classifier_donnees(regression_logistique, 
                                                    parametres['regression_logistique'], 
                                                    x_entrain, y_entrain, x_test, y_test)

TypeError: classifier_donnees() takes 5 positional arguments but 6 were given